In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library
%matplotlib inline

In [35]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [36]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']

In [37]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

--2019-06-11 15:16:46--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2019-06-11 15:16:46--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-11 15:16:47--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following

In [38]:
# Read Toronto cordinates
df_csv = pd.read_csv('Geospatial_Coordinates.csv')

In [39]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [40]:
# Move all Toronto table wiki to data 
data = []
my_string = ''
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    #my_string = ''.join(cols)
    #my_list = my_string.split(",")
    data.append([ele for ele in cols if ele]) # Get rid of empty values

# normalyse data
for data_index in range(len(data)):
    if len(data[data_index]) == 3:
        postcode = data[data_index][0]
        borough = data[data_index][1]
        if borough != 'Not assigned':
            neighbourghood = data[data_index][2]
            if neighbourghood == 'Not assigned':
                neighbourghood = borough
        if data_index>0:
            for partial_index in range(data_index):
                if len(data[partial_index]) == 3:
                    if ((data[partial_index][0] == postcode) and (data[partial_index][1] == borough)):
                        data[partial_index][2] = data[partial_index][2] + ', ' + neighbourghood
                        data[data_index][1] = 'Not assigned' # Mark as 'Not assigned' for future remove

# Create data frame  aliasing as pd and move from data negbourghoods to df

df = pd.DataFrame()
df = pd.DataFrame(columns=['Postal Code','Borough', 'Neighbourghood'])

for data_index in range(len(data)):
    if len(data[data_index]) == 3:
        postcode = data[data_index][0]
        borough = data[data_index][1]
        if borough != 'Not assigned':
            neighbourghood = data[data_index][2]
            if neighbourghood == 'Not assigned':
                neighbourghood = borough
            #  Append a new value to dataframe
            df = df.append({'Postal Code' : postcode , 'Borough' : borough, 'Neighbourghood' : neighbourghood} , ignore_index=True)



In [41]:
# Add coordinates to Toronto Neighbourghood
neighborhoods = pd.merge(df, df_csv, on='Postal Code',how='left')


Loop through the data and fill the dataframe one row at a time to append New York data to Toronto.

In [42]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Postal Code' : 'NYC',
                                          'Borough': borough,
                                          'Neighbourghood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods

,Postal Code,Borough,Neighbourghood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
